# Testing Feature Extraction for OCC ROI Data

In [ ]:
# Imports
import os
import glob
import argparse

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.tri as tri

from scipy.spatial import Delaunay
from scipy.spatial import distance
from scipy.spatial import ConvexHull
from scipy import stats
from shapely.geometry import LineString
from shapely.geometry import Polygon

In [ ]:
# Replace parser args
geoms_dir = os.path.join('..', 'data', 'roi_geoms')
features_dir = os.path.join('..', 'data', 'roi_feats')

In [ ]:
# Descriptive statistics feature calculation
def descriptive_stats(x):
    minimum = np.amin(x)
    maximum = np.amax(x)
    mean = np.mean(x)
    variance = np.var(x)
    standard_deviation = np.std(x)
    skewness = stats.skew(x)
    kurtosis = stats.kurtosis(x)
    moment_5 = stats.moment(x, moment = 5)
    moment_6 = stats.moment(x, moment = 6)
    moment_7 = stats.moment(x, moment = 7)
    moment_8 = stats.moment(x, moment = 8)
    moment_9 = stats.moment(x, moment = 9)
    moment_10 = stats.moment(x, moment = 10)
    moment_11 = stats.moment(x, moment = 11)
    geometric_mean = stats.gmean(x)
    harmonic_mean = stats.hmean(x)
    features = [minimum, maximum, mean, variance, standard_deviation,\
                skewness, kurtosis, moment_5, moment_6, moment_7,\
                moment_8, moment_9, moment_10, moment_11, geometric_mean, harmonic_mean]
    return(features)

In [ ]:
# Create the output directory if it does not exist
if not os.path.exists(features_dir): 
    os.makedirs(features_dir)

In [ ]:
# Locate the geoms and set up directories
geom_files = glob.glob(os.path.join(geoms_dir, '*.npz'))
assert len(geom_files)>0, f'Could not find any geoms in directory {geoms_dir}!'

In [ ]:
# Pull out a single file for testing
geom_file = geom_files[3]

In [ ]:
geom_name = os.path.basename(geom_file)
feature_name = geom_name.replace('.npz', '_features.npz')
feature_path = os.path.join(features_dir, feature_name)

In [ ]:
# Check to see if the features exist
if os.path.exists(feature_path):
    print(f'\tFeature {feature_name} exists at {feature_path}, skipping')

In [ ]:
# Load the geom and check that it is valid for feature extraction
with np.load(geom_file, allow_pickle=True) as f:
    sat_bounds = f['sat_bounds']
    sat_centroids = f['sat_centroids']
    sat_areas = f['sat_areas']
    tum_bounds = f['tum_bounds']
    
    
if len(sat_bounds) == 0:
    print('\tSat bounds is empty!')
if len(sat_centroids) == 0:
    print('\tSat centroids are empty!')
if len(sat_areas) == 0:
    print('\tSat areas are empty!')
if len(sat_centroids) <3:
    print('\tNot enough satellites!')
if len(tum_bounds) == 0:
    print('\ttumor boundaries are empty!')

In [ ]:
# Display geometry properties
print(f'Shape of sat_bounds: {sat_bounds.shape}')
print(f'Shape of sat_centroids: {sat_centroids.shape}')
print(f'Shape of sat_areas: {sat_areas.shape}')
print(f'Shape of tum_bounds: {tum_bounds.shape}')
# DEBUG: Create a function to map the boundaries onto the labelmaps / original slide data

In [ ]:
sat_areas

# Satellite Distance Features

- For each satellite, calculate the distance from each centroid to each tumor boundary point
- Take the minimum distance of that set -- after all sats are processed, we should have 25 minimum values
- Put those mins into the descriptive_statistics function -- yields 16 features

In [ ]:
sat_min_distances = np.amin(distance.cdist(sat_centroids, tum_bounds, 'euclidean'), axis=1)
sat_min_distances_features = descriptive_stats(sat_min_distances)

# Delaunay Triangulation Features

In [ ]:
# Calculate initial triangulation
tri = Delaunay(sat_centroids)

## Eliminate triangle edges that cross the tumor boundary

To do this, we need to:

- Find the simplices of the triangle with respect to the tumor boundary
- Find which of those simplices are greater than 0 (unique and non-negative)
- Create a copy of the simplices and then delete which simplices cross the tumor boundary

We are then left with (a) the satellite centroids, and (b) a set of triangles that include those points which do NOT cross the tumor boundary.

In [ ]:
# Because of the way the tumor and satellite vertices were created, we need to
# rearrange the X and Y coordinates of the tumor boundary parts
tum_bounds = np.transpose(np.vstack((tum_bounds[:,1], tum_bounds[:,0])))

In [ ]:
# Grab the simplices
eliminate_triangles = tri.find_simplex(tum_bounds)
print(f'Length of simplexes that cross the tumor boundary: {len(eliminate_triangles)}')

In [ ]:
# Grab the unique simplices that are greater than -1
eliminate_triangles = np.unique(eliminate_triangles[eliminate_triangles>0])
print(f'Unique, non-negative simplex coordinates: {eliminate_triangles}')

In [ ]:
# Extract the triangles that are listed in "eliminate_triangles"
tri_simplices = tri.simplices.copy()
tri_simplices = np.delete(tri_simplices, eliminate_triangles, axis=0)

In [ ]:
plt.triplot(sat_centroids[:,0], sat_centroids[:,1], tri_simplices)
plt.plot(sat_centroids[:,0], sat_centroids[:,1], 'o')
plt.scatter(tum_bounds[:,0], tum_bounds[:,1])
plt.show()

## Calculate Triangle Length Features

In [ ]:
t = tri.Triangulation(sat_centroids[:,0], sat_centroids[:,1], tri_simplices)

In [ ]:
triangle_lengths = []

for edge in t.edges:
    x1 = sat_centroids[edge[0], 0]
    x2 = sat_centroids[edge[1], 0]
    y1 = sat_centroids[edge[0], 1]
    y2 = sat_centroids[edge[1], 1]
    triangle_lengths.append( np.sqrt((x2-x1)**2 + (y2-y1)**2 ) )

triangle_length_features = descriptive_stats(triangle_lengths)

In [ ]:
plt.hist(triangle_lengths)
plt.show()

## Calculate Triangle Area Features

In [ ]:
triangle_areas = []
for simplex in tri_simplices:
    # Pull out the points for this triangle
    p1 = sat_centroids[simplex[0], :]
    p2 = sat_centroids[simplex[1], :]
    p3 = sat_centroids[simplex[2], :]
    
    # Calculate edge lengths for this triangle
    e12 = np.sqrt( (p2[0]-p1[0])**2 + (p2[1]-p1[1])**2 )
    e13 = np.sqrt( (p3[0]-p1[0])**2 + (p3[1]-p1[1])**2 )
    e23 = np.sqrt( (p3[0]-p2[0])**2 + (p3[1]-p2[1])**2 )
    
    # Calculate area for this triangle
    s = (e12 + e13 + e23) / 2
    a = np.sqrt( s * (s-e12) * (s-e13) * (s-e23))
    triangle_areas.append(a)
    
triangle_area_features = descriptive_stats(triangle_areas)

# Convex Hull / Dispersion Area

The goal of these features is to calculate the relative sat area vs. area of "spread" away from the tumor. 

To do this, we will first calculate:

1. Satellite areas
2. Area of convex hull of satellite areas
3. Difference between the two

In [ ]:
# Calculate satellite areas
plt.scatter(tum_bounds[:,0], tum_bounds[:,1], c='b')
plt.scatter(sat_bounds[:,1], sat_bounds[:,0], c='g')
plt.show()

In [ ]:
satellite_areas = []
for 
sat_hull = ConvexHull(sat_bounds)

In [ ]:
# Convex Hull
points = np.concatenate((sat_bounds, tum_bounds), axis = 0)
hull = ConvexHull(points)
sat_hull = ConvexHull(sat_bounds)
tum_hull = ConvexHull(tum_bounds)
plt.figure(2)
plt.plot(points[hull.vertices,0], points[hull.vertices,1], c='g')
plt.plot(sat_bounds[sat_hull.vertices, 0], sat_bounds[sat_hull.vertices, 1], c='k')
plt.plot(tum_bounds[tum_hull.vertices, 0], tum_bounds[tum_hull.vertices, 1], c='k')
for simplex in sat_hull.simplices:
    print(simplex)
    plt.scatter(sat_bounds[simplex,0], sat_bounds[simplex,1], c='k', alpha=0.5)
plt.show()
for simplex in tum_hull.simplices:
    plt.scatter(tum_bounds[simplex,0], tum_bounds[simplex,1], c='b', alpha=0.5)
plt.show()
for simplex in hull.simplices:
    plt.scatter(points[simplex,0], points[simplex,1], c='r', alpha=0.5)
plt.show()

Area = (hull.volume)-(sat_hull.volume)-(tum_hull.volume)
print(f'Space between tumor and satellites: {Area}')
print(f'Total hull area: {hull.volume}')
print(f'Satellite hull area: {sat_hull.volume}')
print(f'Tumor hull area: {tum_hull.volume}')

tum_area = Polygon([[p[0],p[1]] for p in tum_bounds])
print(f'Tumor area: {tum_area.area}')


# Try "volume" here instead of "area" -- unsure of the difference 
Area = hull.volume- (sat_hull.volume+ tum_hull.volume)
print(Area)
print(f"hull area: {hull.volume}")
print(f"sat area: {sat_hull.volume}")
print(f"tum area: {tum_hull.volume}")


In [ ]:
# Area and Length of sides of each triangle in Delaunay
T_points = sat_centroids[tri.simplices]
tri_edges = []
tri_area = []
for T_point in T_points:
    t1 = LineString([T_point[0,:], T_point[1,:]])
    t2 = LineString([T_point[1,:], T_point[2,:]])
    t3 = LineString([T_point[2,:], T_point[0,:]])
    t4 = Polygon([T_point[0,:], T_point[1,:], T_point[2,:]])
    t_area = t4.area
    t_len = [t1.length, t2.length, t3.length]
    tri_edges.append(t_len)
    tri_area.append(t_area)
    #tri_edge_feats = descriptive_stats(tri_edges)
tri_area_feats = descriptive_stats(tri_area)
